In [1]:
# coding=utf-8
import torch.autograd
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
from torchvision import datasets
from torchvision.utils import save_image
import os

In [2]:
# 创建文件夹
if not os.path.exists('./img'):
    os.mkdir('./img')

In [3]:
def to_img(x):
    out = 0.5 * (x + 1)
    out = out.clamp(0, 1)  # Clamp函数可以将随机变化的数值限制在一个给定的区间[min, max]内：
    out = out.view(-1, 1, 28, 28)  # view()函数作用是将一个多行的Tensor,拼接成一行
    return out

In [4]:
batch_size = 128
num_epoch = 100
z_dimension = 100
# 图像预处理
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # (x-mean) / std
])

In [5]:
# mnist dataset mnist数据集下载
mnist = datasets.MNIST(
    root='./data/', train=True, transform=img_transform, download=True
)

In [6]:
# data loader 数据载入
dataloader = torch.utils.data.DataLoader(
    dataset=mnist, batch_size=batch_size, shuffle=True
)

In [7]:
# 定义判别器  #####Discriminator######使用多层网络来作为判别器
# 将图片28x28展开成784，然后通过多层感知器，中间经过斜率设置为0.2的LeakyReLU激活函数，
# 最后接sigmoid激活函数得到一个0到1之间的概率进行二分类。
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        self.dis = nn.Sequential(
            nn.Linear(784, 256),  # 输入特征数为784，输出为256
            nn.LeakyReLU(0.2),  # 进行非线性映射
            nn.Linear(256, 256),  # 进行一个线性映射
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()  # 也是一个激活函数，二分类问题中，
            # sigmoid可以班实数映射到【0,1】，作为概率值，
            # 多分类用softmax函数
        )
 
    def forward(self, x):
        x = self.dis(x)
        return x

In [8]:
# ###### 定义生成器 Generator #####
# 输入一个100维的0～1之间的高斯分布，然后通过第一层线性变换将其映射到256维,
# 然后通过LeakyReLU激活函数，接着进行一个线性变换，再经过一个LeakyReLU激活函数，
# 然后经过线性变换将其变成784维，最后经过Tanh激活函数是希望生成的假的图片数据分布
# 能够在-1～1之间。
class generator(nn.Module):
    def __init__(self):
        super(generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(100, 256),  # 用线性变换将输入映射到256维
            nn.ReLU(True),  # relu激活
            nn.Linear(256, 256),  # 线性变换
            nn.ReLU(True),  # relu激活
            nn.Linear(256, 784),  # 线性变换
            nn.Tanh()  # Tanh激活使得生成数据分布在【-1,1】之间，因为输入的真实数据的经过transforms之后也是这个分布
        )
 
    def forward(self, x):
        x = self.gen(x)
        return x

In [9]:
# 创建对象
D = discriminator()
G = generator()
if torch.cuda.is_available():
    D = D.cuda()
    G = G.cuda()

In [10]:
# 首先需要定义loss的度量方式  （二分类的交叉熵）
# 其次定义 优化函数,优化函数的学习率为0.0003
criterion = nn.BCELoss()  # 是单目标二分类交叉熵函数
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0003)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0003)

In [ ]:
# ##########################进入训练##判别器的判断过程#####################
for epoch in range(num_epoch):  # 进行多个epoch的训练
    for i, (img, _) in enumerate(dataloader):
        num_img = img.size(0)#图像张数
        # view()函数作用是将一个多行的Tensor,拼接成一行
        # 第一个参数是要拼接的tensor,第二个参数是-1
        # =============================训练判别器==================
        img = img.view(num_img, -1)  # 将图片展开为28*28=784
        real_img = Variable(img).cuda()  # 将tensor变成Variable放入计算图中
        real_label = Variable(torch.ones(num_img)).cuda()  # 定义真实的图片label为1
        fake_label = Variable(torch.zeros(num_img)).cuda()  # 定义假的图片的label为0
 
        # ########判别器训练train#####################
        # 分为两部分：1、真的图像判别为真；2、假的图像判别为假
        # 计算真实图片的损失
        real_out = D(real_img)  # 将真实图片放入判别器中
        d_loss_real = criterion(real_out, real_label)  # 得到真实图片的loss
        real_scores = real_out  # 得到真实图片的判别值，输出的值越接近1越好
        # 计算假的图片的损失
        z = Variable(torch.randn(num_img, z_dimension)).cuda()  # 随机生成一些噪声
        fake_img = G(z).detach()  # 随机噪声放入生成网络中，生成一张假的图片。 # 避免梯度传到G，因为G不用更新, detach分离
        fake_out = D(fake_img)  # 判别器判断假的图片，
        d_loss_fake = criterion(fake_out, fake_label)  # 得到假的图片的loss
        fake_scores = fake_out  # 得到假图片的判别值，对于判别器来说，假图片的损失越接近0越好
        # 损失函数和优化
        d_loss = d_loss_real + d_loss_fake  # 损失包括判真损失和判假损失
        d_optimizer.zero_grad()  # 在反向传播之前，先将梯度归0
        d_loss.backward()  # 将误差反向传播
        d_optimizer.step()  # 更新参数
 
        # ==================训练生成器============================
        # ###############################生成网络的训练###############################
        # 原理：目的是希望生成的假的图片被判别器判断为真的图片，
        # 在此过程中，将判别器固定，将假的图片传入判别器的结果与真实的label对应，
        # 反向传播更新的参数是生成网络里面的参数，
        # 这样可以通过更新生成网络里面的参数，来训练网络，使得生成的图片让判别器以为是真的
        # 这样就达到了对抗的目的
        # 计算假的图片的损失
        z = Variable(torch.randn(num_img, z_dimension)).cuda()  # 得到随机噪声
        fake_img = G(z)  # 随机噪声输入到生成器中，得到一副假的图片
        output = D(fake_img)  # 经过判别器得到的结果
        g_loss = criterion(output, real_label)  # 得到的假的图片与真实的图片的label的loss
        # bp and optimize
        g_optimizer.zero_grad()  # 梯度归0
        g_loss.backward()  # 进行反向传播
        g_optimizer.step()  # .step()一般用在反向传播后面,用于更新生成网络的参数
 
        # 打印中间的损失
        if (i + 1) % 100 == 0:
            print('Epoch[{}/{}],d_loss:{:.6f},g_loss:{:.6f} '
                  'D real: {:.6f},D fake: {:.6f}'.format(
                epoch, num_epoch, d_loss.data.item(), g_loss.data.item(),
                real_scores.data.mean(), fake_scores.data.mean()  # 打印的是真实图片的损失均值
            ))
        if epoch == 0:
            real_images = to_img(real_img.cpu().data)
            save_image(real_images, './img/real_images.png')
    fake_images = to_img(fake_img.cpu().data)
    save_image(fake_images, './img/fake_images-{}.png'.format(epoch + 1))
 
# 保存模型
torch.save(G.state_dict(), './generator.pth')
torch.save(D.state_dict(), './discriminator.pth')

D:\Anaconda3\envs\torch\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch[0/100],d_loss:0.155844,g_loss:3.631157 D real: 0.959836,D fake: 0.104864
Epoch[0/100],d_loss:0.048204,g_loss:4.559946 D real: 0.987232,D fake: 0.034511
Epoch[0/100],d_loss:0.130277,g_loss:5.198682 D real: 0.985142,D fake: 0.096140
Epoch[0/100],d_loss:0.001716,g_loss:8.867710 D real: 0.999589,D fake: 0.001292


D:\Anaconda3\envs\torch\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([96])) that is different to the input size (torch.Size([96, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch[1/100],d_loss:0.071623,g_loss:5.759624 D real: 0.991280,D fake: 0.057365
Epoch[1/100],d_loss:0.041333,g_loss:6.716537 D real: 0.972621,D fake: 0.010407
Epoch[1/100],d_loss:0.260950,g_loss:7.973662 D real: 0.938709,D fake: 0.091862
Epoch[1/100],d_loss:0.777872,g_loss:5.957974 D real: 0.782213,D fake: 0.206278
Epoch[2/100],d_loss:0.532817,g_loss:5.663726 D real: 0.815204,D fake: 0.124831
Epoch[2/100],d_loss:0.247731,g_loss:5.170595 D real: 0.936200,D fake: 0.106136
Epoch[2/100],d_loss:0.558169,g_loss:5.194362 D real: 0.875914,D fake: 0.230737
Epoch[2/100],d_loss:1.288621,g_loss:6.239074 D real: 0.896920,D fake: 0.431277
Epoch[3/100],d_loss:0.263081,g_loss:6.754624 D real: 0.897458,D fake: 0.064383
Epoch[3/100],d_loss:0.475367,g_loss:3.078791 D real: 0.872170,D fake: 0.214283
Epoch[3/100],d_loss:0.926969,g_loss:2.417387 D real: 0.714560,D fake: 0.255041
Epoch[3/100],d_loss:1.182023,g_loss:3.663241 D real: 0.702135,D fake: 0.267183
Epoch[4/100],d_loss:0.969310,g_loss:2.914729 D real:

Epoch[26/100],d_loss:0.443898,g_loss:3.160995 D real: 0.904242,D fake: 0.182467
Epoch[27/100],d_loss:0.292925,g_loss:3.388220 D real: 0.943848,D fake: 0.148240
Epoch[27/100],d_loss:0.326702,g_loss:3.472584 D real: 0.918951,D fake: 0.154397
Epoch[27/100],d_loss:0.225305,g_loss:3.967047 D real: 0.902487,D fake: 0.049139
Epoch[27/100],d_loss:0.351031,g_loss:3.373888 D real: 0.930734,D fake: 0.179123
Epoch[28/100],d_loss:0.603849,g_loss:4.445611 D real: 0.867703,D fake: 0.175066
Epoch[28/100],d_loss:0.511699,g_loss:3.045058 D real: 0.811732,D fake: 0.067053
Epoch[28/100],d_loss:0.351983,g_loss:3.494759 D real: 0.927957,D fake: 0.165841
Epoch[28/100],d_loss:0.438393,g_loss:4.377006 D real: 0.831168,D fake: 0.067394
Epoch[29/100],d_loss:0.508471,g_loss:3.630019 D real: 0.887238,D fake: 0.205499
Epoch[29/100],d_loss:0.438219,g_loss:4.438039 D real: 0.909572,D fake: 0.155431
Epoch[29/100],d_loss:0.661406,g_loss:3.580681 D real: 0.765198,D fake: 0.063369
Epoch[29/100],d_loss:0.201871,g_loss:3.9

Epoch[52/100],d_loss:0.638622,g_loss:2.544138 D real: 0.788485,D fake: 0.202000
Epoch[52/100],d_loss:0.742806,g_loss:2.679699 D real: 0.705744,D fake: 0.113790
Epoch[53/100],d_loss:0.622991,g_loss:2.347201 D real: 0.845778,D fake: 0.236179
Epoch[53/100],d_loss:0.655672,g_loss:1.884527 D real: 0.808857,D fake: 0.237425
Epoch[53/100],d_loss:0.593677,g_loss:2.849642 D real: 0.773252,D fake: 0.125240
Epoch[53/100],d_loss:0.626168,g_loss:2.581709 D real: 0.739133,D fake: 0.125955
Epoch[54/100],d_loss:0.573918,g_loss:2.092688 D real: 0.836067,D fake: 0.222202
Epoch[54/100],d_loss:0.686030,g_loss:2.205938 D real: 0.734425,D fake: 0.090674
Epoch[54/100],d_loss:0.597896,g_loss:2.446588 D real: 0.794839,D fake: 0.172513
Epoch[54/100],d_loss:0.493289,g_loss:2.383564 D real: 0.855210,D fake: 0.192868
Epoch[55/100],d_loss:0.469731,g_loss:2.740997 D real: 0.849743,D fake: 0.165847
Epoch[55/100],d_loss:0.592219,g_loss:2.137535 D real: 0.881702,D fake: 0.249675
Epoch[55/100],d_loss:0.993851,g_loss:1.4

Epoch[78/100],d_loss:0.487776,g_loss:2.202870 D real: 0.824197,D fake: 0.143935
Epoch[78/100],d_loss:0.627007,g_loss:2.428473 D real: 0.840500,D fake: 0.242514
Epoch[78/100],d_loss:0.522263,g_loss:2.455639 D real: 0.801157,D fake: 0.163477
Epoch[79/100],d_loss:0.694517,g_loss:1.808388 D real: 0.836667,D fake: 0.289812
Epoch[79/100],d_loss:0.590673,g_loss:2.623466 D real: 0.751304,D fake: 0.130304
Epoch[79/100],d_loss:0.668016,g_loss:2.124834 D real: 0.786451,D fake: 0.194981
Epoch[79/100],d_loss:0.584355,g_loss:2.263221 D real: 0.794282,D fake: 0.179174
Epoch[80/100],d_loss:0.760850,g_loss:2.331816 D real: 0.750472,D fake: 0.217444
Epoch[80/100],d_loss:0.824719,g_loss:2.181612 D real: 0.691261,D fake: 0.171857
Epoch[80/100],d_loss:0.732930,g_loss:1.661011 D real: 0.805865,D fake: 0.271409
Epoch[80/100],d_loss:0.737900,g_loss:1.659532 D real: 0.779837,D fake: 0.252361
Epoch[81/100],d_loss:0.600727,g_loss:2.931427 D real: 0.810329,D fake: 0.207082
Epoch[81/100],d_loss:0.650307,g_loss:2.1